In [1]:
# !pip install langgraph
# !pip install -U langchain langchain_openai langchainhub tavily-python

In [1]:
from dotenv import load_dotenv, find_dotenv; load_dotenv(find_dotenv())

True

In [3]:
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY=ls__...
# export LANGCHAIN_ENDPOINT=https://api.langchain.plus

### [Define the LangChain Agent](https://python.langchain.com/docs/langgraph#define-the-langchain-agent)

In [11]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

/Users/harukary/Documents/workspace/llm_samples/.venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:160: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


### [Define the nodes](https://python.langchain.com/docs/langgraph#define-the-nodes)

In [5]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish


# Define the agent
# Note that here, we are using `.assign` to add the output of the agent to the dictionary
# This dictionary will be returned from the node
# The reason we don't want to return just the result of `agent_runnable` from this node is
# that we want to continue passing around all the other inputs
agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data.pop('agent_outcome')
    # Get the tool to use
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    # Call that tool on the input
    observation = tool_to_use.invoke(agent_action.tool_input)
    # We now add in the action and the observation to the `intermediate_steps` list
    # This is the list of all previous actions taken and their output
    data['intermediate_steps'].append((agent_action, observation))
    return data

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "exit"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

### [Define the graph](https://python.langchain.com/docs/langgraph#define-the-graph)

In [6]:
from langgraph.graph import END, Graph

workflow = Graph()

# Add the agent node, we give it name `agent` which we will use later
workflow.add_node("agent", agent)
# Add the tools node, we give it name `tools` which we will use later
workflow.add_node("tools", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "tools",
        # Otherwise we finish.
        "exit": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('tools', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
chain = workflow.compile()

### [Use it!](https://python.langchain.com/docs/langgraph#use-it)

In [7]:
res = chain.invoke({"input": "what is the weather in sf", "intermediate_steps": []})
res

{'input': 'what is the weather in sf',
 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}})]),
   [{'url': 'https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/january-1/',
     'content': 'San Francisco Weather in January  San Francisco weather in January San Francisco weather by month // weather averages 9.6 (49.2) 6.2 (43.2) 14 (57.3) 113  Data: 1999 - 2019: avg. Sun hours San Francisco weather and climate for further months  you can find all information about the weather in San Francisco in January:Data: 1991 - 2021 Min. Temperature °C (°F), Max. Temperature °C (°F), Precipitation / Rainfall mm (in),

In [8]:
print(res['agent_outcome'].return_values['output'])

The weather in San Francisco varies throughout the year. In January, the average minimum temperature is 9.6°C (49.2°F) and the average maximum temperature is 14°C (57.3°F). The average precipitation is 113mm. For further information on the weather in San Francisco for other months, you can visit the following link: [San Francisco Weather](https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/january-1/)


### [Streaming Node Output](https://python.langchain.com/docs/langgraph#streaming-node-output)

In [9]:
for output in chain.stream(
    {"input": "what is the weather in sf", "intermediate_steps": []}
):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'input': 'what is the weather in sf', 'intermediate_steps': [], 'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}})])}

---

Output from node 'tools':
---
{'input': 'what is the weather in sf', 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}})]), [{'url': 'https://en.climate-data.org/nor

### [Streaming LLM Tokens](https://python.langchain.com/docs/langgraph#streaming-llm-tokens)

In [10]:
async for output in chain.astream_log(
    {"input": "what is the weather in sf", "intermediate_steps": []},
    include_types=["llm"],
):
    # astream_log() yields the requested logs (here LLMs) in JSONPatch format
    for op in output.ops:
        if op["path"] == "/streamed_output/-":
            # this is the output from .stream()
            ...
        elif op["path"].startswith("/logs/") and op["path"].endswith(
            "/streamed_output/-"
        ):
            # because we chose to only include LLMs, these are LLM tokens
            print(op["value"])

content='' additional_kwargs={'function_call': {'arguments': '', 'name': 'tavily_search_results_json'}}
content='' additional_kwargs={'function_call': {'arguments': '{"', 'name': ''}}
content='' additional_kwargs={'function_call': {'arguments': 'query', 'name': ''}}
content='' additional_kwargs={'function_call': {'arguments': '":"', 'name': ''}}
content='' additional_kwargs={'function_call': {'arguments': 'weather', 'name': ''}}
content='' additional_kwargs={'function_call': {'arguments': ' in', 'name': ''}}
content='' additional_kwargs={'function_call': {'arguments': ' San', 'name': ''}}
content='' additional_kwargs={'function_call': {'arguments': ' Francisco', 'name': ''}}
content='' additional_kwargs={'function_call': {'arguments': '"}', 'name': ''}}
content=''
content=''
content='The'
content=' weather'
content=' in'
content=' San'
content=' Francisco'
content=' varies'
content=' by'
content=' month'
content='.'
content=' In'
content=' January'
content=','
content=' the'
content=' 

### 動作確認

In [17]:
for output in chain.stream(
    {"input": "じゃがいものゆでる、焼く、煮る、蒸す、炒める各調理法を使うレシピをそれぞれ教えてください。また、じゃがいもに対するそれぞれの調理方法のコツを教えてください。それぞれ検索してください。", "intermediate_steps": []}
):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'input': 'じゃがいものゆでる、焼く、煮る、蒸す、炒める各調理法を使うレシピをそれぞれ教えてください。また、じゃがいもに対するそれぞれの調理方法のコツを教えてください。それぞれ検索してください。', 'intermediate_steps': [], 'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'じゃがいものゆでるレシピ'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'じゃがいものゆでるレシピ'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"じゃがいものゆでるレシピ"}', 'name': 'tavily_search_results_json'}})])}

---

Output from node 'tools':
---
{'input': 'じゃがいものゆでる、焼く、煮る、蒸す、炒める各調理法を使うレシピをそれぞれ教えてください。また、じゃがいもに対するそれぞれの調理方法のコツを教えてください。それぞれ検索してください。', 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'じゃがいものゆでるレシピ'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'じゃがいものゆでるレシピ'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"じゃがいものゆでるレシピ"}', 'name': 'tavily_search_r

In [18]:
print(value['agent_outcome'].return_values['output'])

各調理法に対するじゃがいものレシピとコツを以下に示します。

### ゆでる
- レシピ: [じゃがいものゆでるレシピ](https://macaro-ni.jp/80656)
- コツ: じゃがいもをゆでる際は、皮を剥いた状態で水に放り込むとムラなくゆでられます。また、ゆでる時間はじゃがいもの大きさによって異なりますが、フォークなどで刺してみて柔らかければOKです。

### 焼く
- レシピ: [じゃがいもの焼くレシピ](https://www.kitchenbook.jp/topics/1956)
- コツ: じゃがいもを焼く際は、事前に電子レンジで加熱してやわらかくしておくと、中までしっかり火が通ります。また、下味をつけてからオーブントースターでじっくり焼くと、香ばしさが増します。

### 煮る
- レシピ: [じゃがいもの煮るレシピ](https://www.kitchenbook.jp/topics/2234)
- コツ: じゃがいもを煮る際は、野菜や豚ひき肉と一緒に煮込むと、味がよく染み込んで美味しくなります。また、じゃがいもは大きめに切ると、食べ応えがあります。

### 蒸す
- レシピ: [じゃがいもの蒸すレシピ](https://gourmet-note.jp/posts/2893)
- コツ: じゃがいもを蒸す際は、蒸し器を使う方法やレンジを使う方法があります。蒸す時間はじゃがいもの大きさによって異なるため、柔らかさを確認しながら蒸してください。

### 炒める
- レシピ: [じゃがいもの炒めるレシピ](https://delishkitchen.tv/curations/7812)
- コツ: じゃがいもを炒める際は、事前にレンジで加熱してやわらかくしておくと、炒める時間を短縮できます。また、香味野菜と一緒に炒めることで風味豊かな味わいになります。

各レシピを参考に、おいしいじゃがいも料理を楽しんでください！
